# USGS API to map earthquakes on world map

This Jupyter notebook will walk through how to access an API that is available through USGS that details earthquakes over a certain magntiude that occured over a specific time period. The goal of this notebook is to take the USGS hourly/weekly/monthly earthquake RSS feed ( https://earthquake.usgs.gov/earthquakes/feed/) and plot the earthquakes and their relevant magnitudes using a Python Package called Folium (https://python-visualization.github.io/folium/).
 
 ## Requirements
folium  
geopandas  
requests  
json  
numpy

## Install packages

To begin building, we will install first install the Folium Python Package which we will use later on. To install them, we will use the "pip install" framework. The other required packages are already part of Python, so there is no reason to separately install them. 

In [1]:
pip install -q folium

Note: you may need to restart the kernel to use updated packages.


## Import packages

Next to set-up the notebook we will call various packages and modules using the " import function". This will allow us to more seamlessly use the packages as needed throughout the notebook. Packages inclue NumPy (https://numpy.org/) and Pandas (https://pandas.pydata.org/

In [2]:
import requests
import numpy
import json
import pandas as pd
import folium


## Make an API Call 

We then call the USGS API via the the following URL ( details can be found here https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php). Calling the API directly can be helpful as the URL is constantly updated and will provide the most up to date information. This example references the earthquake feed for events that occured in the last month that are over Mw= 2.5. We will convert the url into a data parseable form, JSON (JavaScript Object Notation). 

In [4]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson'
r = requests.get(url)
json_data= r.json()
json_data


{'type': 'FeatureCollection',
 'metadata': {'generated': 1684168224000,
  'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson',
  'title': 'USGS Magnitude 2.5+ Earthquakes, Past Month',
  'status': 200,
  'api': '1.10.3',
  'count': 1722},
 'features': [{'type': 'Feature',
   'properties': {'mag': 4.4,
    'place': 'south of Sumbawa, Indonesia',
    'time': 1684162247632,
    'updated': 1684164448040,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us6000kc1c',
    'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/us6000kc1c.geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'reviewed',
    'tsunami': 0,
    'sig': 298,
    'net': 'us',
    'code': '6000kc1c',
    'ids': ',us6000kc1c,',
    'sources': ',us,',
    'types': ',origin,phase-data,',
    'nst': 23,
    'dmin': 2.668,
    'rms': 0.8,
    'gap': 107,
    'magType': 'mb',
    'type': 'earthquake',
    'tit

## Understanding JSON keys

Once we have the data in the JSON format, we need to identify what data we need from the output to map the data. JSON "keys" are found on the left side of the colons, such as "geometry" and "features". Values for the keys are found on the right side of the colons and give details about those keys. To build a map of the earthquakes, we will pull the coordinate information from the "geometry" key, as well as the title of the earthquake from the "properties key". The values are collected through a "for loop" and are saved for future use. 

In [5]:
lat1 = []
lon1 = []
captions = []
for earthquake in json_data['features']:
    lat,lon,depth= earthquake['geometry']['coordinates']
    longitude=(lon)
    latitude = (lat)
    lat1.append(lat)
    lon1.append(lon)
    labelmarkers= earthquake['properties']['title']
    names=(labelmarkers)
    captions.append(names)


## List of sites and magnitudes 

Using a simple zip function, we are able to put the coordinates and the title/magnitude for the earthquakes into a list.



In [6]:
mapinfo_list  = list (zip(lat1,lon1, captions))

## Dataframe creation 

From the list we are able to convert the information to a dateframe which makes data easier to digest. 



In [9]:
df = pd.DataFrame(mapinfo_list,columns =['latitude','longitude','title'])
df

,latitude,longitude,title
0,117.351100,-11.479000,"M 4.4 - south of Sumbawa, Indonesia"
1,-155.538666,19.160000,"M 2.5 - 7 km SW of Pāhala, Hawaii"
2,-124.746002,40.527000,"M 2.6 - 41km W of Ferndale, CA"
3,-155.277664,19.388834,"M 2.7 - 7 km SW of Volcano, Hawaii"
4,-155.281006,19.385834,"M 2.7 - Island of Hawaii, Hawaii"
...,...,...,...
1717,152.815100,52.173800,"M 4.4 - 264 km WNW of Ozernovskiy, Russia"
1718,-66.779833,17.984833,"M 2.9 - 3 km SSW of Magas Arriba, Puerto Rico"
1719,96.837500,1.164300,"M 4.2 - 87 km W of Gunungsitoli, Indonesia"
1720,-179.724300,-23.480000,M 4.8 - south of the Fiji Islands


## Map Creation 

The final step is to create the map with the earthquake information collated so far. Using Folium documentation (https://python-visualization.github.io/folium/) we will set up a simple map, using locations from the dataframe. 

In [8]:
for index, location_info in df.iterrows():
    my_map=folium.Map(locations=[location_info["longitude"],location_info["latitude"]],zoom_start=10, control_scale=True,tiles= 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                        attr='ESRI')
my_map

 The final folium cell allows the user to add the marker/title and other changes to the base map previously created. Here we added markers and titles to the map. The final map shows the earthquake locations and pop-ups with the name and magintude. Given your project, you could extract various data from the json and create relevant maps/figures with that data.

In [10]:
for index, location_info in df.iterrows():
    folium.Marker([location_info["longitude"], location_info["latitude"]],popup=location_info["title"], color ='purple').add_to(my_map)
my_map